<a href="https://colab.research.google.com/github/Pakapakk/Network_Traffic_Anomaly_Detection/blob/main/anomaly_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [2]:
%%capture
!kaggle datasets download -d galaxyh/kdd-cup-1999-data
! unzip "kdd-cup-1999-data.zip"

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [129]:
kdd_df = pd.read_csv('kddcup.data_10_percent_corrected')

In [130]:
kdd_df

,0,tcp,http,SF,181,5450,0.1,0.2,0.3,0.4,...,9.1,1.00.1,0.00.6,0.11,0.00.7,0.00.8,0.00.9,0.00.10,0.00.11,normal.
0,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.00,0.00,0.00,0.0,0.0,normal.
1,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal.
2,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal.
3,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.00,0.00,0.00,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,59,1.0,0.0,0.02,0.00,0.00,0.00,0.0,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494015,0,tcp,http,SF,310,1881,0,0,0,0,...,255,1.0,0.0,0.01,0.05,0.00,0.01,0.0,0.0,normal.
494016,0,tcp,http,SF,282,2286,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.00,0.01,0.0,0.0,normal.
494017,0,tcp,http,SF,203,1200,0,0,0,0,...,255,1.0,0.0,0.06,0.05,0.06,0.01,0.0,0.0,normal.
494018,0,tcp,http,SF,291,1200,0,0,0,0,...,255,1.0,0.0,0.04,0.05,0.04,0.01,0.0,0.0,normal.


In [131]:
with open("./kddcup.names",'r') as f:
    print(f.read())

back,buffer_overflow,ftp_write,guess_passwd,imap,ipsweep,land,loadmodule,multihop,neptune,nmap,normal,perl,phf,pod,portsweep,rootkit,satan,smurf,spy,teardrop,warezclient,warezmaster.
duration: continuous.
protocol_type: symbolic.
service: symbolic.
flag: symbolic.
src_bytes: continuous.
dst_bytes: continuous.
land: symbolic.
wrong_fragment: continuous.
urgent: continuous.
hot: continuous.
num_failed_logins: continuous.
logged_in: symbolic.
num_compromised: continuous.
root_shell: continuous.
su_attempted: continuous.
num_root: continuous.
num_file_creations: continuous.
num_shells: continuous.
num_access_files: continuous.
num_outbound_cmds: continuous.
is_host_login: symbolic.
is_guest_login: symbolic.
count: continuous.
srv_count: continuous.
serror_rate: continuous.
srv_serror_rate: continuous.
rerror_rate: continuous.
srv_rerror_rate: continuous.
same_srv_rate: continuous.
diff_srv_rate: continuous.
srv_diff_host_rate: continuous.
dst_host_count: continuous.
dst_host_srv_count: con

In [132]:
with open("./training_attack_types",'r') as f:
    print(f.read())

back dos
buffer_overflow u2r
ftp_write r2l
guess_passwd r2l
imap r2l
ipsweep probe
land dos
loadmodule u2r
multihop r2l
neptune dos
nmap probe
perl u2r
phf r2l
pod dos
portsweep probe
rootkit u2r
satan probe
smurf dos
spy r2l
teardrop dos
warezclient r2l
warezmaster r2l




# Data Preprocessing

In [133]:
cols = """
duration,
protocol_type,
service,
flag,
src_bytes,
dst_bytes,
land,
wrong_fragment,
urgent,
hot,
num_failed_logins,
logged_in,
num_compromised,
root_shell,
su_attempted,
num_root,
num_file_creations,
num_shells,
num_access_files,
num_outbound_cmds,
is_host_login,
is_guest_login,
count,
srv_count,
serror_rate,
srv_serror_rate,
rerror_rate,
srv_rerror_rate,
same_srv_rate,
diff_srv_rate,
srv_diff_host_rate,
dst_host_count,
dst_host_srv_count,
dst_host_same_srv_rate,
dst_host_diff_srv_rate,
dst_host_same_src_port_rate,
dst_host_srv_diff_host_rate,
dst_host_serror_rate,
dst_host_srv_serror_rate,
dst_host_rerror_rate,
dst_host_srv_rerror_rate"""

In [134]:
# new_cols = [col.strip() for col in cols.split(',')]
# new_cols.append('target')
# kdd_df.columns = new_cols

In [135]:
attacks_types = {
    'normal': 'normal',
    'back': 'dos',
    'buffer_overflow': 'u2r',
    'ftp_write': 'r2l',
    'guess_passwd': 'r2l',
    'imap': 'r2l',
    'ipsweep': 'probe',
    'land': 'dos',
    'loadmodule': 'u2r',
    'multihop': 'r2l',
    'neptune': 'dos',
    'nmap': 'probe',
    'perl': 'u2r',
    'phf': 'r2l',
    'pod': 'dos',
    'portsweep': 'probe',
    'rootkit': 'u2r',
    'satan': 'probe',
    'smurf': 'dos',
    'spy': 'r2l',
    'teardrop': 'dos',
    'warezclient': 'r2l',
    'warezmaster': 'r2l',
}

In [136]:
# kdd_df['attack_type'] = kdd_df['target'].apply(lambda x: attacks_types[x[:-1]])

In [137]:
kdd_df.head()

,0,tcp,http,SF,181,5450,0.1,0.2,0.3,0.4,...,9.1,1.00.1,0.00.6,0.11,0.00.7,0.00.8,0.00.9,0.00.10,0.00.11,normal.
0,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,59,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


In [138]:
def format_data(kdd_df):
    new_cols = [col.strip() for col in cols.split(',')]
    new_cols.append('target')
    kdd_df.columns = new_cols
    kdd_df['attack_type'] = kdd_df['target'].apply(lambda x: attacks_types[x[:-1]])

In [139]:
format_data(kdd_df)

In [140]:
kdd_df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,target,attack_type
0,0,tcp,http,SF,239,486,0,0,0,0,...,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.,normal
1,0,tcp,http,SF,235,1337,0,0,0,0,...,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.,normal
2,0,tcp,http,SF,219,1337,0,0,0,0,...,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.,normal
3,0,tcp,http,SF,217,2032,0,0,0,0,...,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.,normal
4,0,tcp,http,SF,217,2032,0,0,0,0,...,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.,normal


In [141]:
kdd_df.isnull().sum()

,0
duration,0
protocol_type,0
service,0
flag,0
src_bytes,0
dst_bytes,0
land,0
wrong_fragment,0
urgent,0
hot,0


In [142]:
Kdd_df = kdd_df.dropna(axis='columns')

In [143]:
#Finding categorical features
num_cols = kdd_df._get_numeric_data().columns

cate_cols = list(set(kdd_df.columns)-set(num_cols))
cate_cols.remove('target')
cate_cols.remove('attack_type')

cate_cols

['protocol_type', 'service', 'flag']

remove redundant data (high correlation) to simplify the model without losing significant information.

high correlation (in this context) means that two columns are very similar in terms of their data values.

In [144]:
high_corr_threshold = 0.97

filtered_cols = [col for col in num_cols if col not in ['service', 'protocol_type', 'flag']]

corr_matrix = kdd_df[filtered_cols].corr()
high_corr_pairs = []

for col in corr_matrix.columns:
    related_cols = corr_matrix[col][abs(corr_matrix[col]) > high_corr_threshold].index.tolist()
    if len(related_cols) > 1:
        high_corr_pairs.append((col, related_cols))

keep_columns = set()
drop_columns = set()

grouped_columns = {}
for col, related_cols in high_corr_pairs:
    group = set([col] + related_cols)
    group_key = frozenset(group) # Avoid duplicates by using a frozenset
    if group_key not in grouped_columns:
        grouped_columns[group_key] = list(group)

for group_key, columns in grouped_columns.items():
    columns.sort()
    representative = columns[0]  # Choose the first column as representative
    keep_columns.add(representative)
    for col in columns[1:]:
        drop_columns.add(col)

group_number = 1
for group_key, columns in grouped_columns.items():
    representative = columns[0]
    print(f"Group {group_number} ({representative}-related):")
    print(f"Columns: {', '.join(columns)}")
    print(f"Drop: {', '.join(columns[1:])}")
    print(f"Keep: {representative} (as a representative)\n")
    group_number += 1


Group 1 (num_compromised-related):
Columns: num_compromised, num_root
Drop: num_root
Keep: num_compromised (as a representative)

Group 2 (dst_host_serror_rate-related):
Columns: dst_host_serror_rate, dst_host_srv_serror_rate, serror_rate, srv_serror_rate
Drop: dst_host_srv_serror_rate, serror_rate, srv_serror_rate
Keep: dst_host_serror_rate (as a representative)

Group 3 (dst_host_rerror_rate-related):
Columns: dst_host_rerror_rate, dst_host_srv_rerror_rate, rerror_rate, srv_rerror_rate
Drop: dst_host_srv_rerror_rate, rerror_rate, srv_rerror_rate
Keep: dst_host_rerror_rate (as a representative)

Group 4 (dst_host_same_srv_rate-related):
Columns: dst_host_same_srv_rate, dst_host_srv_count
Drop: dst_host_srv_count
Keep: dst_host_same_srv_rate (as a representative)



In [145]:
kdd_df.shape

(494020, 43)

In [146]:
kdd_df.drop(['num_root'],axis = 1,inplace = True)
kdd_df.drop(['dst_host_srv_serror_rate', 'serror_rate', 'srv_serror_rate'],axis = 1,inplace = True)
kdd_df.drop(['dst_host_srv_rerror_rate', 'rerror_rate', 'srv_rerror_rate'],axis = 1,inplace = True)
kdd_df.drop(['dst_host_srv_count'],axis = 1,inplace = True)

In [147]:
kdd_df.shape

(494020, 35)

In [148]:
kdd_df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,srv_diff_host_rate,dst_host_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_rerror_rate,target,attack_type
0,0,tcp,http,SF,239,486,0,0,0,0,...,0.0,19,1.0,0.0,0.05,0.0,0.0,0.0,normal.,normal
1,0,tcp,http,SF,235,1337,0,0,0,0,...,0.0,29,1.0,0.0,0.03,0.0,0.0,0.0,normal.,normal
2,0,tcp,http,SF,219,1337,0,0,0,0,...,0.0,39,1.0,0.0,0.03,0.0,0.0,0.0,normal.,normal
3,0,tcp,http,SF,217,2032,0,0,0,0,...,0.0,49,1.0,0.0,0.02,0.0,0.0,0.0,normal.,normal
4,0,tcp,http,SF,217,2032,0,0,0,0,...,0.0,59,1.0,0.0,0.02,0.0,0.0,0.0,normal.,normal


In [149]:
# protocol_type feature mapping
pmap = {'icmp': 0, 'tcp': 1, 'udp': 2}
kdd_df['protocol_type'] = kdd_df['protocol_type'].map(pmap)

In [150]:
# flag mapping
fmap = {'SF': 0, 'S0': 1, 'REJ':2, 'RSTR': 3, 'RSTO': 4, 'SH': 5 ,'S1': 6 , 'S2': 7, 'RSTOS0': 8, 'S3': 9 , 'OTH': 10}
kdd_df['flag'] = kdd_df['flag'].map(fmap)

In [151]:
kdd_df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,srv_diff_host_rate,dst_host_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_rerror_rate,target,attack_type
0,0,1,http,0,239,486,0,0,0,0,...,0.0,19,1.0,0.0,0.05,0.0,0.0,0.0,normal.,normal
1,0,1,http,0,235,1337,0,0,0,0,...,0.0,29,1.0,0.0,0.03,0.0,0.0,0.0,normal.,normal
2,0,1,http,0,219,1337,0,0,0,0,...,0.0,39,1.0,0.0,0.03,0.0,0.0,0.0,normal.,normal
3,0,1,http,0,217,2032,0,0,0,0,...,0.0,49,1.0,0.0,0.02,0.0,0.0,0.0,normal.,normal
4,0,1,http,0,217,2032,0,0,0,0,...,0.0,59,1.0,0.0,0.02,0.0,0.0,0.0,normal.,normal


drop target out as we aim to classify the types of atttack

In [152]:
kdd_df['target'].value_counts()

,count
target,
smurf.,280790
neptune.,107201
normal.,97277
back.,2203
satan.,1589
ipsweep.,1247
portsweep.,1040
warezclient.,1020
teardrop.,979


In [153]:
kdd_df['attack_type'].value_counts()

,count
attack_type,
dos,391458
normal,97277
probe,4107
r2l,1126
u2r,52


In [154]:
kdd_df['service'].value_counts()

,count
service,
ecr_i,281400
private,110893
http,64292
smtp,9723
other,7237
...,...
X11,11
tim_i,7
pm_dump,1


In [155]:
kdd_df.drop('service', axis = 1, inplace= True)

In [156]:
kdd_df.head()

,duration,protocol_type,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,...,srv_diff_host_rate,dst_host_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_rerror_rate,target,attack_type
0,0,1,0,239,486,0,0,0,0,0,...,0.0,19,1.0,0.0,0.05,0.0,0.0,0.0,normal.,normal
1,0,1,0,235,1337,0,0,0,0,0,...,0.0,29,1.0,0.0,0.03,0.0,0.0,0.0,normal.,normal
2,0,1,0,219,1337,0,0,0,0,0,...,0.0,39,1.0,0.0,0.03,0.0,0.0,0.0,normal.,normal
3,0,1,0,217,2032,0,0,0,0,0,...,0.0,49,1.0,0.0,0.02,0.0,0.0,0.0,normal.,normal
4,0,1,0,217,2032,0,0,0,0,0,...,0.0,59,1.0,0.0,0.02,0.0,0.0,0.0,normal.,normal


In [157]:
len(kdd_df.columns)

34

# Scaling

In [158]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

In [159]:
kdd_df = kdd_df.drop(['target'], axis=1)

In [160]:
X = kdd_df.drop(['attack_type',], axis=1)
Y = kdd_df[['attack_type']]

In [161]:
minmax_scaler = MinMaxScaler()
X = minmax_scaler.fit_transform(X)

In [162]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1166)

In [163]:
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

(345814, 32) (148206, 32)
(345814, 1) (148206, 1)


# GaussainNB

# Logistic Regression

# KerasClassifier